# Notebook Imports

In [2]:
import pandas as pd
import numpy as np

# Constants

In [3]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into Numpy Array

In [4]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [6]:
sparse_train_data[:5]

array([[ 0,  0,  1,  2],
       [ 0,  7,  1,  1],
       [ 0, 15,  1,  1],
       [ 0, 18,  1,  1],
       [ 0, 24,  1,  1]])

In [7]:
sparse_train_data[-5:]

array([[5795, 1539,    0,    2],
       [5795, 1903,    0,    1],
       [5795, 1915,    0,    2],
       [5795, 2127,    0,    1],
       [5795, 2279,    0,    1]])

In [8]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 265427
Nr of rows in test file 110522


In [9]:
print('Nr of emails in training file', np.unique(sparse_train_data[:, 0]).size)
print('Nr of emails in test file', np.unique(sparse_test_data[:, 0]).size)

Nr of emails in training file 4015
Nr of emails in test file 1724


### How to create an empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [13]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [14]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
sparse_train_data[10:13]

array([[ 0, 71,  1,  4],
       [ 0, 88,  1,  1],
       [ 0, 94,  1,  1]])

In [16]:
sparse_train_data[10][3]

4

## Create a full matrix from a sparse matrix

In [17]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1,
                    cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas DataFrame.
    Keyword arguments:
    sparse matrix - numpy array
    nr_words - size of the vocabulary. Total number of tokens
    doc_idx - position of the document id in the sparse matrix. Default: 1st column
    word_idx - position of the word id in the sparse matrix. Default: 2nd column
    cat_idx - position of the label (spam is 1, non-spam is 0). Default: 3rd column
    freq_idx - positon of occurence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_train_data[:, 0])
    full_matrix = pd.DataFrame(index=index_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [18]:
%%time

full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 12.6 s


In [19]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0


## Calculating the Probaility of Spam

In [20]:
# Challenge: Calculate the probability of spam - the percent of spam messages in the training
# dataset. Store this value in a variable called prob_spam

full_train_data.CATEGORY.size

4015

In [21]:
full_train_data.CATEGORY.sum()

1250

In [22]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is:', prob_spam)

Probability of spam is: 0.31133250311332505


## Total Number of Words/Tokens

In [23]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [24]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4015,)

In [25]:
email_lengths[:5]

DOC_ID
0     50
1     76
2     87
3     76
4    136
dtype: int64

In [26]:
total_word_count = email_lengths.sum()
total_word_count

445891

## Number of Tokens in Spam & Ham Emails

**Challenge:** Create a subset of the email_lengths series that only contains the spam messages. Call the subset spam_lengths.
Then count the total number of words that occur in spam emails.

Do the same for the non-spam emails. Create a subset called ham_lengths. Then count the total umber of words that occur in the ham emails

In [27]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1250,)

In [28]:
spam_word_count = spam_lengths.sum()
spam_word_count

195645

In [29]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [30]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [31]:
nonspam_word_count = ham_lengths.sum()
nonspam_word_count 

250246

In [32]:
spam_word_count + nonspam_word_count - total_word_count

0

In [33]:
# Challenge: Can you figure out if spam emails or non-spam emails tend to be longer?
# Which category contains more tokens ? Take a guess and then verify your intuition using the dataset

print('Average Nr of words in spam emails {:.0f}'.format(spam_word_count/spam_lengths.shape[0]))
print('Average Nr of words in ham emails {:.3f}'.format(nonspam_word_count/ham_lengths.shape[0]))

Average Nr of words in spam emails 157
Average Nr of words in ham emails 90.505


## Summing the Tokens Occuring in Spam

In [34]:
full_train_features.shape

(4015, 2500)

In [35]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [36]:
train_spam_tokens.shape

(1250, 2500)

In [37]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [38]:
# Challenge: Repeat this process for the ham messages. Sum the toens that occur in the 
# non-spam messages. Store the values in a variable called summed_ham_tokens

## Summing the Tokens Occuring in Ham

In [39]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [40]:
summed_ham_tokens.shape

(2500,)

In [41]:
summed_ham_tokens.tail()

2495    13
2496    19
2497     1
2498    16
2499    35
dtype: int64

In [42]:
train_ham_tokens[2499].sum() + 1

35

## P(Token|Spam) - Probability that a Token Occurs given the Email is Spam

In [44]:
prob_tokens_spam = summed_spam_tokens / (spam_word_count + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.009049
1    0.004885
2    0.006838
3    0.010593
4    0.006803
dtype: float64

## P(Token|Ham) - Probability that a Token Occurs given the Email is Non-Spam

In [50]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_word_count + VOCAB_SIZE)
prob_tokens_nonspam[:5]
#prob_tokens_nonspam.sum()

0    0.020863
1    0.009887
2    0.008063
3    0.003735
4    0.006311
dtype: float64

## P(Token) - Probability that a Token Occurs

In [53]:
prob_tokens_all = full_train_features.sum(axis=0) / total_word_count
#prob_tokens_all[:5]
prob_tokens_all.sum()

1.0

## Save the Trained Models

In [54]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

## Prepare Test Data

In [55]:
sparse_test_data.shape

(110522, 4)

In [69]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

Wall time: 2min 36s


In [70]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [72]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)